In [2]:
import cv2





def detection(frame):

    # Load the pre-trained face detection model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))    
    
    x, y, w, h = 0, 0, 0, 0  # Default values
       
    # Iterate through the detected faces, print the coordinates, and draw rectangles
    for (x, y, w, h) in faces:
        print(f'Face detected at coordinates: X={x}, Y={y}, w={w}, h={h}')
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return frame,x,y,w,h

def blur1(frame, x, y, w, h):
    image = frame
    # Increase the box size by 10%
    increase_factor = 0.1
    x -= int(w * increase_factor)
    y -= int(h * increase_factor)
    w += int(2 * w * increase_factor)
    h += int(2 * h * increase_factor)

    # Ensure that the coordinates are within the image boundaries
    x = max(x, 0)
    y = max(y, 0)
    w = min(w, image.shape[1])
    h = min(h, image.shape[0])

    # Extract the region of interest (ROI) based on the coordinates
    face_roi = image[y:y+h, x:x+w]

    # Check if the face_roi is empty (no face detected)
    if face_roi.size == 0:
        return frame

    # Apply Gaussian blur to the face region
    blurred_face = cv2.GaussianBlur(face_roi, (0, 0), sigmaX=30)

    # Replace the original face region with the blurred face
    image[y:y+h, x:x+w] = blurred_face
    return image


def extract_frames(video_path, output_directory, frame_interval):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Initialize variables
    frame_count = 0
    success = True
    next_extraction_time = frame_interval

    # Read the video frames
    while success:
        # Read the next frame
        success, frame = video.read()

        # Check if the frame read was successful
        if success:
            # Get the frame rate of the video
            frame_rate = video.get(cv2.CAP_PROP_FPS)

            # Calculate the time of the current frame
            current_time = frame_count / frame_rate

            # Check if the current frame should be extracted
            if current_time >= next_extraction_time:
                # Create a copy of the frame for face detection
                frame_copy = frame.copy()

                # Generate the output file name
                
                
                face_image_data = detection(frame_copy)
                
                # Check if a face is detected in the frame
                if face_image_data is not None:
                    face_image = face_image_data[0]
                    x = face_image_data[1]
                    y = face_image_data[2]
                    w = face_image_data[3]
                    h = face_image_data[4]
                    
                    blur = blur1(face_image, x, y, w, h)
                
                               
#                 face_image_data = detection(frame_copy)
                
               
#                 face_image = face_image_data[0]
#                 x = face_image_data[1]
#                 y = face_image_data[2]
#                 w = face_image_data[3]
#                 h = face_image_data[4]
                
#                 blur = blur1(face_image,x,y,w,h)
                
                output_file = f"{output_directory}/frame_{frame_count}.jpg"

                # Save the frame as an image file
                cv2.imwrite(output_file, blur)
                # Print the frame information
                print(f"Extracted frame {frame_count} at time {current_time:.2f} seconds")

                # Update the time for the next frame extraction
                next_extraction_time += frame_interval

            # Increment the frame count
            frame_count += 1

    # Release the video object outside the loop
    video.release()

# Example usage
video_path = r"v1.mp4"
output_directory = r"algo1_images"
frame_interval = 0.5  # Extract frames every .5 seconds

extract_frames(video_path, output_directory, frame_interval)


Face detected at coordinates: X=419, Y=98, w=476, h=476
Extracted frame 13 at time 0.52 seconds
Face detected at coordinates: X=472, Y=101, w=435, h=435
Extracted frame 25 at time 1.00 seconds
Face detected at coordinates: X=502, Y=105, w=404, h=404
Extracted frame 38 at time 1.52 seconds
Face detected at coordinates: X=502, Y=107, w=397, h=397
Extracted frame 50 at time 2.00 seconds
Face detected at coordinates: X=515, Y=95, w=408, h=408
Extracted frame 63 at time 2.52 seconds
Face detected at coordinates: X=521, Y=82, w=416, h=416
Extracted frame 75 at time 3.00 seconds
Face detected at coordinates: X=561, Y=96, w=369, h=369
Extracted frame 88 at time 3.52 seconds
Face detected at coordinates: X=570, Y=85, w=368, h=368
Extracted frame 100 at time 4.00 seconds
Face detected at coordinates: X=593, Y=97, w=340, h=340
Extracted frame 113 at time 4.52 seconds
Extracted frame 125 at time 5.00 seconds
Extracted frame 138 at time 5.52 seconds
Face detected at coordinates: X=613, Y=114, w=321

In [22]:
import cv2
import os

def images_to_video(image_folder, output_video, fps):
    # Get the list of image files in the folder
    image_files = [os.path.join(image_folder, img) for img in sorted(os.listdir(image_folder)) if img.endswith(".jpg")]

    # Sort the image files by filename
    image_files.sort()

    if not image_files:
        print("No image files found in the folder.")
        return

    # Read the first image to get the image size
    frame = cv2.imread(image_files[0])
    height, width, layers = frame.shape

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec as needed
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    for image in image_files:
        frame = cv2.imread(image)
        out.write(frame)  # Write the frame to the video

    out.release()
    print(f"Video '{output_video}' created successfully.")

# Specify the folder containing image files, output video file, and frames per second (fps)
image_folder = "algo1_images"  # Replace with your image folder path
output_video = "output_video.mp4"
fps = 30  # Adjust the frame rate as needed

# Call the function to create the video
images_to_video(image_folder, output_video, fps)


Video 'output_video.mp4' created successfully.


# live video

In [3]:
import cv2

In [4]:
cap = cv2.VideoCapture(0)  # Use 0 for the default camera. You can specify a different camera if you have multiple cameras.

In [5]:
cap

< cv2.VideoCapture 0x7fc59e4ece10>

In [6]:
while True:
    ret, frame = cap.read()  # Read a frame from the camera

    if not ret:
        break

    # Perform any image processing or analysis here if needed
    import cv2





    def detection(frame):

        # Load the pre-trained face detection model
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    #     image_path = frame
        # Load an image for detection
    #     image_path = 'face1.jpg'  # Replace with the path to your image
    #     image = cv2.imread(image_path)

        # Convert the image to grayscale (required for the face detection model)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Perform face detection
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))


        x, y, w, h = 0, 0, 0, 0  # Default values


        # Iterate through the detected faces, print the coordinates, and draw rectangles
        for (x, y, w, h) in faces:
            print(f'Face detected at coordinates: X={x}, Y={y}, w={w}, h={h}')
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        return frame,x,y,w,h
        # Display the image with detected faces and coordinates
    #     cv2.imshow('Face Detection', image)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()
    #     cv2.imwrite(output_file, image)


    # image_path = 'frame_13.jpg'
    # detection(image_path)

    def blur1(frame, x, y, w, h):
        image = frame
        # Increase the box size by 10%
        increase_factor = 0.1
        x -= int(w * increase_factor)
        y -= int(h * increase_factor)
        w += int(2 * w * increase_factor)
        h += int(2 * h * increase_factor)

        # Ensure that the coordinates are within the image boundaries
        x = max(x, 0)
        y = max(y, 0)
        w = min(w, image.shape[1])
        h = min(h, image.shape[0])

        # Extract the region of interest (ROI) based on the coordinates
        face_roi = image[y:y+h, x:x+w]

        # Check if the face_roi is empty (no face detected)
        if face_roi.size == 0:
            return frame

        # Apply Gaussian blur to the face region
        blurred_face = cv2.GaussianBlur(face_roi, (0, 0), sigmaX=30)

        # Replace the original face region with the blurred face
        image[y:y+h, x:x+w] = blurred_face
        return image


        # Display the result
    #     cv2.imshow('Blurred Face', image1)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()




    def extract_frames(video_path, output_directory, frame_interval):
        # Open the video file
        video = cv2.VideoCapture(video_path)

        # Initialize variables
        frame_count = 0
        success = True
        next_extraction_time = frame_interval

        # Read the video frames
        while success:
            # Read the next frame
            success, frame = video.read()

            # Check if the frame read was successful
            if success:
                # Get the frame rate of the video
                frame_rate = video.get(cv2.CAP_PROP_FPS)

                # Calculate the time of the current frame
                current_time = frame_count / frame_rate

                # Check if the current frame should be extracted
                if current_time >= next_extraction_time:
                    # Create a copy of the frame for face detection
                    frame_copy = frame.copy()

                    # Generate the output file name


                    face_image_data = detection(frame_copy)

                    # Check if a face is detected in the frame
                    if face_image_data is not None:
                        face_image = face_image_data[0]
                        x = face_image_data[1]
                        y = face_image_data[2]
                        w = face_image_data[3]
                        h = face_image_data[4]

                        blur = blur1(face_image, x, y, w, h)


    #                 face_image_data = detection(frame_copy)


    #                 face_image = face_image_data[0]
    #                 x = face_image_data[1]
    #                 y = face_image_data[2]
    #                 w = face_image_data[3]
    #                 h = face_image_data[4]

    #                 blur = blur1(face_image,x,y,w,h)

                    output_file = f"{output_directory}/frame_{frame_count}.jpg"

                    # Save the frame as an image file
                    cv2.imwrite(output_file, blur)
                    # Print the frame information
                    print(f"Extracted frame {frame_count} at time {current_time:.2f} seconds")

                    # Update the time for the next frame extraction
                    next_extraction_time += frame_interval

                # Increment the frame count
                frame_count += 1

        # Release the video object outside the loop
        video.release()

    # Example usage
    video_path = r"v1.mp4"
    output_directory = r"algo1_images"
    frame_interval = 0.5  # Extract frames every .5 seconds

    extract_frames(video_path, output_directory, frame_interval)


    
    cv2.imshow('Live Video', frame)  # Display the frame in a window named 'Live Video'
    print(frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press the 'Esc' key to exit the loop
        break
        

        
        
        
# cap.release()
# cv2.destroyAllWindows()

[[[14 20 21]
  [14 20 21]
  [16 20 21]
  ...
  [19 19 19]
  [19 19 14]
  [19 19 14]]

 [[15 19 20]
  [15 19 20]
  [15 19 20]
  ...
  [17 19 14]
  [17 19 14]
  [17 19 14]]

 [[15 19 19]
  [13 18 17]
  [13 18 17]
  ...
  [17 19 14]
  [21 20 15]
  [21 20 15]]

 ...

 [[21 18 19]
  [21 18 19]
  [21 17 20]
  ...
  [20 20 20]
  [19 19 19]
  [17 17 17]]

 [[22 21 24]
  [22 21 24]
  [22 19 20]
  ...
  [20 19 21]
  [20 19 21]
  [19 18 20]]

 [[23 21 21]
  [22 19 20]
  [20 19 21]
  ...
  [19 18 20]
  [19 18 20]
  [19 18 20]]]
[[[29 17 19]
  [29 17 19]
  [27 17 19]
  ...
  [23 17 27]
  [23 14 34]
  [23 14 34]]

 [[24 15 16]
  [26 16 17]
  [23 18 19]
  ...
  [20 17 26]
  [22 15 28]
  [22 15 28]]

 [[23 18 19]
  [23 18 19]
  [21 18 19]
  ...
  [20 16 28]
  [19 15 27]
  [19 15 27]]

 ...

 [[16 16 16]
  [17 17 17]
  [17 17 17]
  ...
  [19 19 19]
  [19 20 15]
  [19 20 15]]

 [[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [19 19 19]
  [19 19 19]
  [19 19 19]]

 [[16 16 16]
  [17 17 17]
  [17 17 17]
  .

[[[27 19 14]
  [27 19 14]
  [19 20 15]
  ...
  [30 21 22]
  [30 21 22]
  [30 21 22]]

 [[21 19 13]
  [21 19 13]
  [17 17 17]
  ...
  [25 20 21]
  [28 18 20]
  [27 17 19]]

 [[17 19 14]
  [16 18 13]
  [17 19 14]
  ...
  [23 18 19]
  [23 18 19]
  [23 18 19]]

 ...

 [[21 17 17]
  [20 16 16]
  [20 16 16]
  ...
  [21 21 21]
  [22 20 25]
  [21 19 24]]

 [[23 17 20]
  [20 15 18]
  [18 16 16]
  ...
  [24 21 24]
  [24 20 25]
  [24 20 25]]

 [[21 17 17]
  [20 16 16]
  [19 16 19]
  ...
  [24 21 24]
  [24 20 25]
  [24 20 25]]]
[[[19 19 19]
  [19 19 19]
  [19 19 19]
  ...
  [16 16 16]
  [16 16 16]
  [16 16 16]]

 [[19 19 19]
  [19 19 19]
  [19 19 19]
  ...
  [17 17 17]
  [17 17 17]
  [16 16 16]]

 [[19 18 20]
  [19 18 20]
  [19 18 20]
  ...
  [19 19 19]
  [19 19 19]
  [17 17 17]]

 ...

 [[15 21 15]
  [15 21 15]
  [19 19 19]
  ...
  [19 19 19]
  [19 18 20]
  [17 17 19]]

 [[15 21 15]
  [15 21 15]
  [15 19 19]
  ...
  [19 18 20]
  [19 18 20]
  [16 15 18]]

 [[13 20 14]
  [13 20 14]
  [13 20 14]
  .

[[[35 18 21]
  [36 19 22]
  [30 21 22]
  ...
  [17 18 29]
  [21 17 29]
  [21 17 29]]

 [[30 21 22]
  [30 21 22]
  [26 21 24]
  ...
  [18 19 30]
  [22 19 28]
  [22 19 28]]

 [[23 19 24]
  [22 18 23]
  [22 18 23]
  ...
  [18 20 28]
  [18 21 25]
  [17 20 24]]

 ...

 [[15 19 19]
  [15 19 19]
  [13 20 19]
  ...
  [20 19 21]
  [20 19 21]
  [20 19 21]]

 [[10 17 16]
  [13 20 19]
  [13 18 17]
  ...
  [20 19 21]
  [20 20 20]
  [19 19 19]]

 [[12 17 16]
  [12 17 16]
  [13 18 17]
  ...
  [20 20 20]
  [21 18 19]
  [21 18 19]]]
[[[40 19 22]
  [40 19 22]
  [30 18 20]
  ...
  [30 15 26]
  [24 18 28]
  [25 20 30]]

 [[35 20 18]
  [34 19 17]
  [29 17 19]
  ...
  [37 17 23]
  [24 20 25]
  [24 20 25]]

 [[35 17 15]
  [35 17 15]
  [29 18 15]
  ...
  [27 15 22]
  [22 19 21]
  [21 17 20]]

 ...

 [[19 18 20]
  [20 19 21]
  [20 20 20]
  ...
  [20 16 16]
  [20 16 16]
  [19 14 15]]

 [[15 19 19]
  [16 21 20]
  [16 21 20]
  ...
  [21 17 17]
  [19 17 17]
  [18 16 16]]

 [[15 19 19]
  [15 19 19]
  [15 19 19]
  .

[[[24 13 33]
  [25 14 34]
  [22 14 31]
  ...
  [17 24 23]
  [22 18 30]
  [22 18 30]]

 [[22 13 33]
  [23 14 34]
  [25 14 32]
  ...
  [15 22 21]
  [16 22 25]
  [15 20 24]]

 [[24 13 33]
  [25 14 34]
  [25 17 27]
  ...
  [15 19 19]
  [16 20 21]
  [16 20 21]]

 ...

 [[22 22 22]
  [21 21 21]
  [21 21 21]
  ...
  [24 22 22]
  [23 21 21]
  [23 21 21]]

 [[16 21 20]
  [16 21 20]
  [22 22 22]
  ...
  [21 18 19]
  [21 20 15]
  [21 20 15]]

 [[16 21 20]
  [16 21 20]
  [20 20 20]
  ...
  [19 19 14]
  [21 20 15]
  [19 19 14]]]
[[[12 17 16]
  [15 19 19]
  [13 20 19]
  ...
  [30 22 19]
  [30 20 24]
  [30 20 24]]

 [[13 18 17]
  [13 18 17]
  [13 18 17]
  ...
  [28 18 20]
  [29 19 21]
  [28 18 20]]

 [[15 19 19]
  [15 19 19]
  [13 18 17]
  ...
  [24 19 20]
  [27 17 19]
  [27 17 19]]

 ...

 [[21 17 20]
  [21 17 20]
  [21 17 20]
  ...
  [21 18 19]
  [21 18 19]
  [21 18 19]]

 [[21 17 22]
  [21 17 22]
  [23 16 22]
  ...
  [22 20 25]
  [23 21 21]
  [22 19 20]]

 [[22 18 23]
  [22 18 23]
  [24 17 23]
  .

In [2]:
# import cv2

# def detection(frame):

#     # Load the pre-trained face detection model
#     face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Perform face detection
#     faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))    
    
#     x, y, w, h = 0, 0, 0, 0  # Default values
       
#     # Iterate through the detected faces, print the coordinates, and draw rectangles
#     for (x, y, w, h) in faces:
# #         print(f'Face detected at coordinates: X={x}, Y={y}, w={w}, h={h}')
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
#     return frame,x,y,w,h


# def blur1(frame, x, y, w, h):
#     image = frame
#     # Increase the box size by 10%
#     increase_factor = 0.05
#     x -= int(w * increase_factor)
#     y -= int(h * increase_factor)
#     w += int(2 * w * increase_factor)
#     h += int(2 * h * increase_factor)

#     # Ensure that the coordinates are within the image boundaries
#     x = max(x, 0)
#     y = max(y, 0)
#     w = min(w, image.shape[1])
#     h = min(h, image.shape[0])

#     # Extract the region of interest (ROI) based on the coordinates
#     face_roi = image[y:y+h, x:x+w]

#     # Check if the face_roi is empty (no face detected)
#     if face_roi.size == 0:
#         return frame

#     # Apply Gaussian blur to the face region
#     blurred_face = cv2.GaussianBlur(face_roi, (0, 0), sigmaX=30)

#     # Replace the original face region with the blurred face
#     image[y:y+h, x:x+w] = blurred_face
#     return image


# # Initialize the video capture device
# cap = cv2.VideoCapture(0)

# while True:
#     ret, frame = cap.read()  # Read a frame from the camera

#     if not ret:
#         break

#     # Perform face detection and get the face region
#     face_image_data = detection(frame)

#     if face_image_data is not None:
#         face_image = face_image_data[0]
#         x, y, w, h = face_image_data[1:]

#         # Apply your blur function to the detected face region
#         blurred_frame = blur2(frame, x, y, w, h)

#         # Display the frame with the blurring effect
#         cv2.imshow('Live Video', blurred_frame)
#     else:
#         cv2.imshow('Live Video', frame)

#     if cv2.waitKey(1) & 0xFF == 27:  # Press the 'Esc' key to exit the loop
#         break

# # Release the camera and close any open windows when done
# cap.release()
# cv2.destroyAllWindows()

error: OpenCV(4.8.1) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


# @@@@@@@@@@@@@@@Final Code@@@@@@@@@@@@@@@

In [1]:
def detection(frame):

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))    
    x, y, w, h = 0, 0, 0, 0  # Default values
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return frame,x,y,w,h

In [2]:
def blur1(frame, x, y, w, h):
    image = frame
    increase_factor = 0.1
    x -= int(w * increase_factor)
    y -= int(h * increase_factor)
    w += int(2 * w * increase_factor)
    h += int(2 * h * increase_factor)

    # Ensure that the coordinates are within the image boundaries
    x = max(x, 0)
    y = max(y, 0)
    w = min(w, image.shape[1])
    h = min(h, image.shape[0])

    # Extract the region of interest (ROI) based on the coordinates
    face_roi = image[y:y+h, x:x+w]

    # Check if the face_roi is empty (no face detected)
    if face_roi.size == 0:
        return frame

    blurred_face = cv2.GaussianBlur(face_roi, (0, 0), sigmaX=30)
    image[y:y+h, x:x+w] = blurred_face
    return image

In [3]:
def blur2(frame,x,y,w,h):
    
    image = frame
    increase_factor = 0.05
    x -= int(w * increase_factor)
    y -= int(h * increase_factor)
    w += int(2 * w * increase_factor)
    h += int(2 * h * increase_factor)

    # Ensure that the coordinates are within the image boundaries
    x = max(x, 0)
    y = max(y, 0)
    w = min(w, image.shape[1])
    h = min(h, image.shape[0])

    face_roi = image[y:y+h, x:x+w]

    block_size = 20 
    small_face = cv2.resize(face_roi, (w // block_size, h // block_size), interpolation=cv2.INTER_LINEAR)
    pixelated_face = cv2.resize(small_face, (w, h), interpolation=cv2.INTER_NEAREST)
    image[y:y+h, x:x+w] = pixelated_face
    return image

In [4]:
def blur3(frame,x,y,w,h): 
    image = frame
    # Increase the box size by 10%
    increase_factor = 0.05
    x -= int(w * increase_factor)
    y -= int(h * increase_factor)
    w += int(2 * w * increase_factor)
    h += int(2 * h * increase_factor)

    x = max(x, 0)
    y = max(y, 0)
    w = min(w, image.shape[1])
    h = min(h, image.shape[0])

    face_roi = image[y:y+h, x:x+w]
    block_size = 25  
    small_face = cv2.resize(face_roi, (w // block_size, h // block_size), interpolation=cv2.INTER_LINEAR)
    mosaic_face = cv2.resize(small_face, (w, h), interpolation=cv2.INTER_NEAREST)
    image[y:y+h, x:x+w] = mosaic_face

    return image

In [6]:
import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read() 

    if not ret:
        break

    face_image_data = detection(frame)
    
    if face_image_data is not None:
        face_image = face_image_data[0]
        x, y, w, h = face_image_data[1:]

        if w > 0 and h > 0: 
            blurred_frame = blur2(frame, x, y, w, h)
            cv2.imshow('Live Video', blurred_frame)
        else:
            cv2.imshow('Live Video', frame)
    else:
        cv2.imshow('Live Video', frame)
    if cv2.waitKey(1) & 0xFF == 27:  
        break

cap.release()
cv2.destroyAllWindows()

In [5]:
# import cv2
# import tkinter as tk
# from PIL import Image, ImageTk

# # Initialize the video capture device
# cap = cv2.VideoCapture(0)

# # Create a Tkinter window
# root = tk.Tk()
# root.title("Live Video Streams")

# # Create four labels to display the live video feeds
# original_label = tk.Label(root)
# original_label.pack(side="left")

# blur1_label = tk.Label(root)
# blur1_label.pack(side="left")

# blur2_label = tk.Label(root)
# blur2_label.pack(side="left")

# blur3_label = tk.Label(root)
# blur3_label.pack(side="left")

# while True:
#     ret, frame = cap.read()  # Read a frame from the camera

#     if not ret:
#         break

#     # Perform face detection and get the face region
#     frame, x, y, w, h = detection(frame)

#     if w > 0 and h > 0:  # Check that the detected face region is not empty
#         # Apply your blur functions to the detected face region
#         blurred_frame1 = blur1(frame, x, y, w, h)
#         blurred_frame2 = blur2(frame, x, y, w, h)
#         blurred_frame3 = blur3(frame, x, y, w, h)

#         # Convert the frames to RGB format for displaying with Tkinter
#         original_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         blur1_image = cv2.cvtColor(blurred_frame1, cv2.COLOR_BGR2RGB)
#         blur2_image = cv2.cvtColor(blurred_frame2, cv2.COLOR_BGR2RGB)
#         blur3_image = cv2.cvtColor(blurred_frame3, cv2.COLOR_BGR2RGB)

#         # Create PhotoImage objects to display in the labels
#         original_photo = ImageTk.PhotoImage(Image.fromarray(original_image))
#         blur1_photo = ImageTk.PhotoImage(Image.fromarray(blur1_image))
#         blur2_photo = ImageTk.PhotoImage(Image.fromarray(blur2_image))
#         blur3_photo = ImageTk.PhotoImage(Image.fromarray(blur3_image))

#         original_label.configure(image=original_photo)
#         blur1_label.configure(image=blur1_photo)
#         blur2_label.configure(image=blur2_photo)
#         blur3_label.configure(image=blur3_photo)

#         # Update the window
#         root.update()

#     if cv2.waitKey(1) & 0xFF == 27:  # Press the 'Esc' key to exit the loop
#         break

# # Release the camera and close the Tkinter window when done
# cap.release()
# root.destroy()


RuntimeError: Too early to create image: no default root window

In [ ]:
# def detection(frame):

#     # Load the pre-trained face detection model
#     face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#     image_path = frame
#     # Load an image for detection
# #     image_path = 'face1.jpg'  # Replace with the path to your image
#     image = cv2.imread(image_path)

#     # Convert the image to grayscale (required for the face detection model)
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Perform face detection
#     faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

#     # Iterate through the detected faces, print the coordinates, and draw rectangles
#     for (x, y, w, h) in faces:
#         print(f'Face detected at coordinates: X={x}, Y={y}, w={w}, h={h}')
#         cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

#     # Display the image with detected faces and coordinates
#     cv2.imshow('Face Detection', image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

# detection("frame_jpg")



# # Increase the box size by 10%
# increase_factor = 0.1
# x -= int(w * increase_factor)
# y -= int(h * increase_factor)
# w += int(2 * w * increase_factor)
# h += int(2 * h * increase_factor)

# # Ensure that the coordinates are within the image boundaries
# x = max(x, 0)
# y = max(y, 0)
# w = min(w, image.shape[1])
# h = min(h, image.shape[0])

# # Extract the region of interest (ROI) based on the coordinates
# face_roi = image[y:y+h, x:x+w]

# # Apply Gaussian blur to the face region
# blurred_face = cv2.GaussianBlur(face_roi, (0, 0), sigmaX=30)

# # Replace the original face region with the blurred face
# image[y:y+h, x:x+w] = blurred_face
# image1 = image
# # Display the result
# cv2.imshow('Blurred Face', image1)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [7]:
# def detection(frame):

#     # Load the pre-trained face detection model
#     face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#     image_path = frame
#     # Load an image for detection
# #     image_path = 'face1.jpg'  # Replace with the path to your image
#     image = cv2.imread(image_path)

#     # Convert the image to grayscale (required for the face detection model)
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Perform face detection
#     faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

#     # Iterate through the detected faces, print the coordinates, and draw rectangles
#     for (x, y, w, h) in faces:
#         print(f'Face detected at coordinates: X={x}, Y={y}, w={w}, h={h}')
#         cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# #     # Display the image with detected faces and coordinates
# #     cv2.imshow('Face Detection', image)
# #     cv2.waitKey(0)
# #     cv2.destroyAllWindows()
    
    
# image_path = 'frame_13.jpg'
# detection(image_path)


Face detected at coordinates: X=414, Y=93, w=487, h=487


In [10]:
# image_path = 'frame_13.jpg'
# image = cv2.imread(image_path)
# image


array([[[140, 118, 112],
        [141, 119, 113],
        [141, 119, 113],
        ...,
        [103,  84,  81],
        [102,  83,  80],
        [102,  83,  80]],

       [[139, 117, 111],
        [140, 118, 112],
        [140, 118, 112],
        ...,
        [103,  84,  81],
        [102,  83,  80],
        [102,  83,  80]],

       [[139, 117, 111],
        [139, 117, 111],
        [140, 118, 112],
        ...,
        [102,  83,  80],
        [102,  83,  80],
        [102,  83,  80]],

       ...,

       [[ 99,  84,  81],
        [ 99,  84,  81],
        [ 99,  84,  81],
        ...,
        [ 69,  61,  62],
        [ 68,  60,  61],
        [ 68,  60,  61]],

       [[ 99,  84,  81],
        [ 99,  84,  81],
        [ 99,  84,  81],
        ...,
        [ 69,  61,  62],
        [ 69,  61,  62],
        [ 68,  60,  61]],

       [[ 99,  84,  81],
        [ 99,  84,  81],
        [ 99,  84,  81],
        ...,
        [ 69,  61,  62],
        [ 70,  62,  63],
        [ 69,  61,  62]]

In [ ]:
# def blur1(image):
#     # Increase the box size by 10%
#     increase_factor = 0.1
#     x -= int(w * increase_factor)
#     y -= int(h * increase_factor)
#     w += int(2 * w * increase_factor)
#     h += int(2 * h * increase_factor)

#     # Ensure that the coordinates are within the image boundaries
#     x = max(x, 0)
#     y = max(y, 0)
#     w = min(w, image.shape[1])
#     h = min(h, image.shape[0])

#     # Extract the region of interest (ROI) based on the coordinates
#     face_roi = image[y:y+h, x:x+w]

#     # Apply Gaussian blur to the face region
#     blurred_face = cv2.GaussianBlur(face_roi, (0, 0), sigmaX=30)

#     # Replace the original face region with the blurred face
#     image[y:y+h, x:x+w] = blurred_face
#     image1 = image
#     # Display the result
#     cv2.imshow('Blurred Face', image1)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()


In [8]:
# def detection(frame):

#     # Load the pre-trained face detection model
#     image_path = frame

#     face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#     # Load an image for detection
# #     image_path = 'face1.jpg'  # Replace with the path to your image
#     frame = cv2.imread(image_path)

#     # Convert the image to grayscale (required for the face detection model)
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Perform face detection
#     faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

#     # Iterate through the detected faces, print the coordinates, and draw rectangles
#     for (x, y, w, h) in faces:
#         print(f'Face detected at coordinates: X={x}, Y={y}, w={w}, h={h}')
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
#     return frame,x,y,w,h



In [10]:
# image_path = 'frame_13.jpg'
# a = detection(image_path)

Face detected at coordinates: X=414, Y=93, w=487, h=487


In [16]:
a[4]

487